# System rekomendacji filmów

W tym notebooku wczytujemy dane o ocenach użytkowników oraz informacje o filmach (w tym gatunki). Następnie ładujemy wytrenowany model sieci neuronowej i mapowania ID.

Dla wybranego użytkownika wyświetlamy filmy, które ocenił, oraz generujemy 10 najlepszych rekomendacji filmów, które model przewiduje, że mogą mu się spodobać.



In [4]:
!pip install -r requirements_predict.txt

In [5]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [6]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model



# Wczytanie danych
ratings = pd.read_csv('ratings.dat', sep='::', engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'],
                      encoding='latin1')

movies = pd.read_csv('movies.dat', sep='::', engine='python',
                     names=['movieId', 'title', 'genres'],
                     encoding='latin1')
movies['main_genre'] = movies['genres'].str.split('|').str[0]

# maping
with open('user2user_encoded.pkl', 'rb') as f:
    user2user_encoded = pickle.load(f)
with open('movie2movie_encoded.pkl', 'rb') as f:
    movie2movie_encoded = pickle.load(f)
with open('genre2genre_encoded.pkl', 'rb') as f:
    genre2genre_encoded = pickle.load(f)
with open('user_encoded2user.pkl', 'rb') as f:
    user_encoded2user = pickle.load(f)
with open('movie_encoded2movie.pkl', 'rb') as f:
    movie_encoded2movie = pickle.load(f)

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Wczytanie modelu
model = load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

# Pobranie ID gatunku
def get_genre_id_safe(main_genre):
    return genre2genre_encoded.get(main_genre, 0)

#  Funkcja rekomendująca
def predict_top_movies(user_id, top_n=10):
    if user_id not in user2user_encoded:
        print(" Za mało filmów obejrzanych")
        return pd.DataFrame()

    encoded_user_id = user2user_encoded[user_id]

    ratings['user_encoded'] = ratings['userId'].map(user2user_encoded)
    ratings['movie_encoded'] = ratings['movieId'].map(movie2movie_encoded)

    # Filmy ocenione przez tego użytkownika
    watched_encoded = ratings[ratings['user_encoded'] == encoded_user_id]['movie_encoded'].tolist()

    # Kandydaci do rekomendacji
    all_encoded_movies = list(movie2movie_encoded.values())
    candidates = [m for m in all_encoded_movies if m not in watched_encoded]

    if not candidates:
        print(" Brak kandydatów do rekomendacji.")
        return pd.DataFrame()

    # Przygotowanie danych wejściowych
    encoded_movies = np.array(candidates)
    encoded_genres = np.array([
        get_genre_id_safe(movies[movies['movieId'] == movie_encoded2movie[m]]['main_genre'].values[0])
        for m in encoded_movies
    ])
    user_array = np.full(len(encoded_movies), encoded_user_id)

    preds = model.predict([user_array, encoded_movies, encoded_genres], verbose=0).flatten()
    top_indices = preds.argsort()[-top_n:][::-1]

    top_encoded = encoded_movies[top_indices]
    top_preds = preds[top_indices]
    top_movie_ids = [movie_encoded2movie[eid] for eid in top_encoded]

    top_df = movies[movies['movieId'].isin(top_movie_ids)][['movieId', 'title', 'genres']]
    top_df = top_df.set_index('movieId').loc[top_movie_ids].reset_index()
    top_df['predicted_rating'] = np.clip(top_preds, 0.5, 5.0)

    return top_df


# Test rekomendacji dla człowieka

user_id = 109

# Wyświetlenie filmów ocenionych przez użytkownika, zobaczenie czy obejrzał wystarczająca ilosc iflmow
if user_id in user2user_encoded:
    encoded_user = user2user_encoded[user_id]
    ratings['user_encoded'] = ratings['userId'].map(user2user_encoded)
    rated_movies = ratings[ratings['user_encoded'] == encoded_user][['movieId', 'rating']]
    watched = rated_movies.merge(movies[['movieId', 'title']], on='movieId', how='left')
    print(f"\n🎞 Filmy ocenione przez użytkownika {user_id}:")
    print(watched[['title', 'rating']].to_string(index=False))
else:
    print("\n Użytkownik obejrzał za mało filmów.")

# Wyświetlenie rekomendacji
recommended = predict_top_movies(user_id, top_n=20)
print(f"\n Rekomendacje dla użytkownika {user_id}:")
print(recommended.to_string(index=False))

#Jeśli chciałbys zobaczyc oceny dla siebie pwinnienes dodac je do zbioru dancyh potem wytrenowac na nich






🎞 Filmy ocenione przez użytkownika 109:
                                                                      title  rating
                                            Honey, I Shrunk the Kids (1989)       4
                     Naked Gun: From the Files of Police Squad!, The (1988)       5
                                                           Toy Story (1995)       4
                                                             Jumanji (1995)       4
                                                     Doctor Dolittle (1967)       4
                                                Nutty Professor, The (1963)       4
                                     One Flew Over the Cuckoo's Nest (1975)       2
                                                Arsenic and Old Lace (1944)       4
                      Star Wars: Episode V - The Empire Strikes Back (1980)       4
                                             Raiders of the Lost Ark (1981)       5
                                   